# 自编码器

In [1]:
import tensorflow as tf
import numpy as np
import sklearn.preprocessing as prep

In [2]:
from tensorflow.examples.tutorials.mnist import  input_data

In [3]:
class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self, n_input, n_hidden, transfer_function = tf.nn.softplus,
                optimizer = tf.train.AdamOptimizer(), scale = 0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale
        network_weights = self._initialize_weights()
        self.weights = network_weights

        self.x = tf.placeholder(tf.float32,[None,self.n_input])
        self.hidden = self.transfer(tf.add(tf.matmul(self.x + self.training_scale 
                                                  * tf.random_normal((n_input,)),
                                                  self.weights['w1']),self.weights['b1']))
        self.reconstruction = tf.add(tf.matmul(self.hidden,self.weights['w2']),self.weights['b2'])

        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction,self.x),2.0))
        self.optimizer = optimizer.minimize(self.cost)

        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
    
    def _initialize_weights(self):
        all_weights = {}
        all_weights['w1'] = tf.get_variable('w1',shape=[self.n_input,self.n_hidden],
                                            initializer=tf.contrib.layers.xavier_initializer())
        all_weights['b1'] = tf.Variable(tf.constant(0.0, shape=[self.n_hidden]))

        all_weights['w2'] = tf.get_variable('w2',shape=[self.n_hidden,self.n_input],
                                            initializer=tf.contrib.layers.xavier_initializer())
        all_weights['b2'] = tf.Variable(tf.constant(0.0, shape=[self.n_input]))
        
        return all_weights
    
    def partial_fit(self,X):
        cost, opt = self.sess.run((self.cost, self.optimizer),
                                 feed_dict = {self.x: X, self.scale: self.training_scale})
        return cost

    def calc_total_cost(self,X):
        return self.sess.run(self.cost, 
                             feed_dict = {self.x: X, self.scale: self.training_scale})
    
    def transform(self,X):
        return self.sess.run(self,hidden, 
                             feed_dict = {self.x: X, self.scale: self.training_scale})
    def generate(self,hidden = None):
        if isinstance(hidden,NoneType):
            hidden = np.random.normal(size = self.weights['b1'])
        return self.sess.run(self.reconstruction,
                            feed_dict = {self.hidden: hidden})
    def reconstruct(self,X):
        return self.sess.run(self.reconstruction,
                            feed_dict = {self.x: X, self.scale: self.training_scale})
    
    def getWeights(self):
        return self.sess.run(self.weights['w1'])
    
    def getBiases(self):
        return self.sess.run(self.weights['b1'])
    

In [4]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
def standard_scale(X_train,X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train,X_test

In [6]:
def get_random_block_from_data(data,batch_size):
    start_index = np.random.randint(0,len(data)-batch_size)
    return data[start_index:(start_index+batch_size)]

In [7]:
X_train, X_test = standard_scale(mnist.train.images, mnist.test.images)
n_samples = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step = 1

In [8]:
autoencoder = AdditiveGaussianNoiseAutoencoder(n_input = 784, n_hidden = 200,
                                              transfer_function = tf.nn.softplus,
                                              optimizer = tf.train.AdamOptimizer(.001),
                                              scale = .01)

In [9]:
for  epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train,batch_size)
        
        cost = autoencoder.partial_fit(batch_xs)
        avg_cost += cost/n_samples*batch_size
        
    if epoch % display_step == 0:
        print("Epoch:",'%04d' % (epoch+1),"cost = ","{:.9f}".format(avg_cost))

('Epoch:', '0001', 'cost = ', '24821.000206818')
('Epoch:', '0002', 'cost = ', '13652.216509091')
('Epoch:', '0003', 'cost = ', '11467.686970455')
('Epoch:', '0004', 'cost = ', '10536.909668182')
('Epoch:', '0005', 'cost = ', '9906.749840341')
('Epoch:', '0006', 'cost = ', '9181.882810227')
('Epoch:', '0007', 'cost = ', '8777.622546023')
('Epoch:', '0008', 'cost = ', '9086.523168750')
('Epoch:', '0009', 'cost = ', '8245.600673295')
('Epoch:', '0010', 'cost = ', '8141.477089773')
('Epoch:', '0011', 'cost = ', '8364.602891477')
('Epoch:', '0012', 'cost = ', '7925.563444318')
('Epoch:', '0013', 'cost = ', '8673.351383523')
('Epoch:', '0014', 'cost = ', '7593.315647727')
('Epoch:', '0015', 'cost = ', '8531.950359659')
('Epoch:', '0016', 'cost = ', '8298.646916477')
('Epoch:', '0017', 'cost = ', '8219.743362500')
('Epoch:', '0018', 'cost = ', '8236.716220455')
('Epoch:', '0019', 'cost = ', '7820.585081250')
('Epoch:', '0020', 'cost = ', '7754.361401136')
